https://docs.fast.ai/text.transform.html
https://github.com/fastai/fastai/blob/0b31610c6a836c56a337e2a34ee2d1510456d1c6/tests/test_text_transform.py#L19

1. Load the dataset into a pandas dataframe.
1. Use regular expressions to remove elements that are not words such as HTML tags, LaTeX expressions, URLs, digits, line returns, and so on.
1. Remove missing values for texts
1. Remove texts that are extremely large or too short to bring any information to the model. We want to keep paragraphs that contain at least a few words and remove the paragraphs that are composed of large numerical tables.
1. Use a tokenizer to create a version of the original text that is a string of space-separated lowercase tokens.

   Download the dataset

In [20]:
import os.path
from os import path
from urllib import request

GZ_FILE   = 'stackexchange_812k.csv.gz'
#DATA_FILE = 'stackexchange_812k.csv'
DATA_URL  = 'https://liveproject-resources.s3.amazonaws.com/116/other/stackexchange_812k.csv.gz'
if not path.exists(f'data/{GZ_FILE}'):
    request.urlretrieve(DATA_URL, f'data/{GZ_FILE}')          
    

In [2]:
from fastai.text import *

In [4]:
tokenizer = Tokenizer(BaseTokenizer)

In [5]:
texts = ['one two three four', 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.', "I'm suddenly SHOUTING FOR NO REASON"]


In [6]:
tokenizer.process_all(texts)

[['one', 'two', 'three', 'four'],
 ['xxmaj',
  'lorem',
  'ipsum',
  'dolor',
  'sit',
  'amet,',
  'consectetur',
  'adipiscing',
  'elit,',
  'sed',
  'do',
  'eiusmod',
  'tempor',
  'incididunt',
  'ut',
  'labore',
  'et',
  'dolore',
  'magna',
  'aliqua.'],
 ['xxmaj',
  "i'm",
  'suddenly',
  'xxup',
  'shouting',
  'xxup',
  'for',
  'xxup',
  'no',
  'xxup',
  'reason']]

load the dataset into a pandas dataframe

In [26]:
import gzip
import pandas as pd
with gzip.open(f'data/{GZ_FILE}') as f:
   df = pd.read_csv(f)

In [27]:
df.head()

,post_id,parent_id,comment_id,text,category
0,1,NaN,NaN,Eliciting priors from experts,title
1,2,NaN,NaN,What is normality?,title
2,3,NaN,NaN,What are some valuable Statistical Analysis op...,title
3,4,NaN,NaN,Assessing the significance of differences in d...,title
4,6,NaN,NaN,The Two Cultures: statistics vs. machine learn...,title


Use regular expressions to remove elements that are not words such as HTML tags, LaTeX expressions, URLs, digits, line returns, and so on.


In [15]:
#what will fastai do here?
html = fix_html("<b>hello</b>") #didn't really fix it. 
tokenizer.process_all([html])

[['<b>hello<', '/', 'b>']]

Remove missing values for texts